# Libraries

In [4]:
import pandas as pd
import numpy as np

from collections import defaultdict
import re
import nltk

from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer

In [5]:
sample=10

# Step 1: Data

In [6]:
airbnb_data=pd.read_csv("Airbnb_Texas_Rentals.csv",usecols=['average_rate_per_night', 'bedrooms_count', 'city',
       'date_of_listing', 'description', 'latitude', 'longitude', 'title','url'],parse_dates=['date_of_listing'])

In [7]:
airbnb_data.columns

Index(['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing',
       'description', 'latitude', 'longitude', 'title', 'url'],
      dtype='object')

# Step 2: Create documents

In [8]:
airbnb_data.head(1)

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,2016-05-01,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...


In [9]:
airbnb_data.shape

(18259, 9)

# Clean data

In [10]:
# Check null values of the dataset
airbnb_data.isnull().sum()
#average_rate_per_night -> replace NAN with 0, convert to int
#bedrooms_count -> There are only 3 records so we decided to replace NAN with a category based on the desciption if it's possible. 
#description, latitude, longitude, title -> replace NAN to 'Unknown'

average_rate_per_night    28
bedrooms_count             3
city                       0
date_of_listing            0
description                2
latitude                  34
longitude                 34
title                      3
url                        0
dtype: int64

In [11]:
airbnb_data.dtypes

average_rate_per_night            object
bedrooms_count                    object
city                              object
date_of_listing           datetime64[ns]
description                       object
latitude                         float64
longitude                        float64
title                             object
url                               object
dtype: object

In [12]:
def clean(airbnb_data):
    """
    Method that removes nan values and imputes them
    
    Input: dataframe
    Output: cleaned dataframe
    
    """
    #replace NAN with 0
    airbnb_data.average_rate_per_night.replace(np.nan, '$0',inplace=True)
    #convert to int and remove $
    airbnb_data.average_rate_per_night=airbnb_data.average_rate_per_night.replace('[\$]', '', regex=True).astype(int)

    #replace NAN with'unknown'

    airbnb_data.description.replace(np.nan,'unknown',inplace=True)
    airbnb_data.title.replace(np.nan,'unknown',inplace=True)

    airbnb_data.latitude.replace(np.nan,'unknown',inplace=True)
    airbnb_data.longitude.replace(np.nan,'unknown',inplace=True)

    #check where bedrooms_count doesn't have a value and save indexes of those records to a list
    null_value_idx=airbnb_data[airbnb_data.bedrooms_count.isnull()].index
    #if the word studio is mentioned in description then it is a studio otherwise 'unknown'
    for idx in null_value_idx:
        if 'studio' in airbnb_data.iloc[idx].description.split():
            airbnb_data.bedrooms_count[idx]='Studio'
        else:
            airbnb_data.bedrooms_count[idx]='unknown'
        
    return airbnb_data

In [13]:
airbnb_data=clean(airbnb_data)
airbnb_data.isnull().sum()

C:\Users\Dusica\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dusica\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


average_rate_per_night    0
bedrooms_count            0
city                      0
date_of_listing           0
description               0
latitude                  0
longitude                 0
title                     0
url                       0
dtype: int64

In [14]:
airbnb_data.shape

(18259, 9)

In [15]:
def create_tsv_documents(airbnb_data):
    """
    Method that creates different .tsv files for each record in the airbnb_data 
    
    Input: dataframe
    """   
    #clean data
    airbnb_data=clean(airbnb_data)
    
    #for each index make a dataframe of airbnb_data and store it into new tsv file
    for i in airbnb_data.index:
        pd.DataFrame(airbnb_data.loc[i]).transpose().to_csv('data/doc_'+str(i)+'.tsv',sep='\t')

#method is run only once at the beginning to make separate .tsv files
create_tsv_documents(airbnb_data)

# Preprocessing

1) Removing stop words

2) Removing punctuation

3) Stemming

##### remove non english words and words Giulia chooses (room, price, airbnb) MOST often ones_?
##### should we remove numbers__?

# 3.1) Conjunctive query

## 3.1.1) Create your index!

In [16]:
def preprocessing_text(df):
    #remove upper cases
    df=df.lower()
    #replacing new line sign '\n' with a whitespace ' '    
    df=df.replace('\\n',' ')

    #removing stop words and punctuation
    stop_words = set(stopwords.words('english')) 

    #for removing punctuations
    tokenizer = RegexpTokenizer(r'\w+')
    
    #to tokenize the string
    word_tokens = tokenizer.tokenize(df) 

    #stemming
    ps = PorterStemmer()
    filtered_words = [ps.stem(w) for w in word_tokens if not w in stop_words] 

    #remove non-english words
    
    return filtered_words

In [17]:
#Building a vocabulary

#set for vocabulary (values of the set will be the keys fo vocabulary_dict)
vocabulary_lst=[]
#building a dictionary which will be used for making an inverted index
doc_vocabs=defaultdict(list)

for i in range(sample):
    #take one file
    df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
    #preprocessing 
    df=df.description[0]+' '+df.title[0]+' '+df.city[0]
    filtered_words=preprocessing_text(df)
    temp_vocabulary_set=set()
    for word in filtered_words:
        temp_vocabulary_set.add(word)
    vocabulary_lst.append(temp_vocabulary_set)
    doc_vocabs[i]=list(temp_vocabulary_set)
vocabulary_set=set.union(*vocabulary_lst)

In [18]:
len(vocabulary_set)

194

In [19]:
vocabulary={}
for k,v in enumerate(vocabulary_set):
    #just for testing
    #vocabulary[v]='id'+str(k)
    vocabulary[v]= k

In [20]:
len(vocabulary)

194

In [21]:
def save_vocabulary(vocabulary): 
    """
    method that converts vocabulary into a dataframe and saves it into a csv file
    
    input: vocabulary(dictionary, key='term',value='term_id')
    """
    vocabulary_dataframe=pd.DataFrame()
    vocabulary_dataframe['word']=vocabulary.keys()
    vocabulary_dataframe.to_csv('vocabulary_sample.csv')
    del vocabulary_dataframe

In [22]:
save_vocabulary(vocabulary)

# Compute an inverted index

In [23]:
def compute_inverted_idx(doc_vocabs,vocabulary):
    """
    method that computes an inverted index
    
    input: doc_vocabs(dictionary), vocabulary(dictionary of all unique words, key=term, value=term_id)
    output: inverted_idx(dictionary, key=term_id, value=list of document_ids) 
    """
    #initialize defaultdict for making an inverted index
    inverted_idx = defaultdict(list)
    #in every document look for every word and assign document id to the words which belong to it
    for idx in doc_vocabs.keys():
        for word in doc_vocabs[idx]:
            inverted_idx[vocabulary[word]].append(idx)
    return inverted_idx

In [24]:
inverted_idx=compute_inverted_idx(doc_vocabs,vocabulary)

In [25]:
# Save a dictionary into a pickle file.
import pickle

pickle.dump(inverted_idx, open("inv_idx_sample.p", "wb"))  # save it into a file named save.p

# Load the dictionary back from the pickle file.

inverted_index = pickle.load(open("inv_idx_sample.p", "rb"))

# 3.1.2) Execute the query

In [26]:
def finalize_output(result_set):
    df=pd.DataFrame()
    for i,val in enumerate(result_set):
        pd.set_option('display.max_colwidth', -1)
        df=df.append(pd.read_csv('data/doc_'+str(val)+'.tsv',sep='\t',usecols=['description','title','city','url']
                                 ,encoding='ISO-8859-1',index_col=False))
        df.reset_index().drop('index',axis=1)
    return df

In [27]:
def search_engine():
    user_query=str(input())
    
    user_query=preprocessing_text(user_query)

    list_term_idx=[]
    result_set=[]
    for word in user_query:
        #if word exist in the vocabulary
        if word in vocabulary.keys():
            list_term_idx.append(set(inverted_idx[vocabulary[word]]))
        else:
            list_term_idx.append({'x'})
            break
    result_set=list(set.intersection(*list_term_idx))
    if 'x' in result_set or not result_set:
        result_set='No results! Try again!'
        return result_set
        
    print(result_set)
    result_set=finalize_output(result_set)
    return result_set

search_engine()

In [28]:
#TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it) 

# 3.2) Conjunctive query & Ranking score
### 3.2.1) Inverted index
### 3.2.2) Execute the query

# calculate tfldf

1) tf=term frequency -- the frequency of the word in each document in the corpus.

2) idf


In [29]:
#Calc tfidf and cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Calculate IDF

In [30]:
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)
Total_number_of_documents=airbnb_data.shape[0]

In [31]:
#TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it) 

In [32]:
# TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
tf_dic=dict()
for i in range(sample):
    #take one file
    df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
    #preprocessing 
    df=df.description[0]+' '+df.title[0]+' '+df.city[0]
    filtered_words=preprocessing_text(df)
    tf_series=pd.Series(filtered_words)
    tf_series=(tf_series.value_counts())/len(tf_series)
    for index,value in tf_series.iteritems():
        tf_dic[index,i]=value

In [33]:
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)
idf_dic=dict()
total_num_docs_sample=sample
for i in range(sample):
    #take one file
    df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
    #preprocessing 
    df=df.description[0]+' '+df.title[0]+' '+df.city[0]
    
    filtered_words=preprocessing_text(df)
 #   for word_k in filtered_words_2: 
    idf_series=pd.Series(list(set(filtered_words)))
    idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])) )
    for idx in range(len(idf_series)):
        idf_dic[idf_series[idx],i]=idf_calc[idx]      
        
  #      lis.append(np.log(total_num_docs_sample/len(inverted_idx[vocabulary[word_k]])))
 

In [34]:
df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
#preprocessing 
df=df.description[0]+' '+df.title[0]+' '+df.city[0]
filtered_words=preprocessing_text(df)

tf_series=pd.Series(filtered_words)
tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
idf_series=pd.Series(list(set(filtered_words))).sort_values()
idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
#idf_calc

In [35]:
#idf_calc.sort_index()
result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),idf_calc],axis=1)#.reset_index()
result_df['res']=result_df[1]*result_df[2]

In [36]:
result_df

,0,1,2,res
0,antonio,0.125,0.916291,0.114536
1,cozi,0.125,1.609438,0.201180
2,entranc,0.125,1.609438,0.201180
3,histor,0.125,2.302585,0.287823
4,privat,0.250,1.203973,0.300993
5,san,0.125,2.302585,0.287823
6,studio,0.125,2.302585,0.287823


In [37]:
result_df

,0,1,2,res
0,antonio,0.125,0.916291,0.114536
1,cozi,0.125,1.609438,0.201180
2,entranc,0.125,1.609438,0.201180
3,histor,0.125,2.302585,0.287823
4,privat,0.250,1.203973,0.300993
5,san,0.125,2.302585,0.287823
6,studio,0.125,2.302585,0.287823


In [38]:
# TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
tf_idf_dic=dict()
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)
idf_dic2={}
tf_dic2={}
proba={}
#OVDE
total_num_docs_sample=sample
result_df=pd.DataFrame()
for i in range(sample):
    #take one file
    df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
    #preprocessing 
    df=df.description[0]+' '+df.title[0]+' '+df.city[0]
    filtered_words=preprocessing_text(df)
    tf_series=pd.Series(filtered_words)
    tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
    idf_series=pd.Series(list(set(filtered_words))).sort_values()
    idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
    #idf_calc.sort_index()
    result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),pd.Series(idf_calc.values)],axis=1)#.reset_index()
#    result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),idf_calc],axis=1)#.reset_index()
    result_df['tf_idf']=result_df[1]*result_df[2]
   # result_df=result_df.loc[:,['res',0]]
    
    for idx in range(result_df.shape[0]):
        tf_idf_dic[result_df[0][idx],i]=result_df['tf_idf'][idx]
#del result_df
    for idx in range(len(tf_series)):
#        tf_idf_dic[tf_series.index[idx],i]=tf_series[idx]*idf_calc[idx]
        idf_dic2[idf_series[idx],i]=idf_calc[idx] 
    for index,value in tf_series.iteritems():
        tf_dic2[index,i]=value
    for k in tf_dic2.keys():
        proba[k]=tf_dic2[k]*idf_dic2[k]

In [39]:
result_df

,0,1,2,tf_idf
0,antonio,0.125,1.609438,0.201180
1,cozi,0.125,1.609438,0.201180
2,entranc,0.125,2.302585,0.287823
3,histor,0.125,2.302585,0.287823
4,privat,0.250,0.916291,0.229073
5,san,0.125,1.203973,0.150497
6,studio,0.125,2.302585,0.287823


In [40]:
idf_calc.values

array([1.60943791, 1.60943791, 2.30258509, 2.30258509, 0.91629073,
       1.2039728 , 2.30258509])

In [41]:
for key in tf_dic.keys():
    if(tf_dic[key]*idf_dic[key]!=proba[key]):
 #       print(key,tf_dic[key])
#        print(key,idf_dic[key])
        print(key,tf_dic[key]*idf_dic[key])
        print(key,tf_idf_dic[key])
        print("!!!!!!!!!!!!!!!!!!!!!!!")

In [42]:
proba['san',2]

0.05733203830123505

In [43]:
for key in tf_dic.keys():
    if(tf_dic[key]*idf_dic[key]!=tf_idf_dic[key]):
 #       print(key,tf_dic[key])
#        print(key,idf_dic[key])
        print(key,tf_dic[key]*idf_dic[key])
        print(key,tf_idf_dic[key])
        print("!!!!!!!!!!!!!!!!!!!!!!!")

In [44]:
# First way
#TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

def calculate_tf_idf(airbnb_data):
    tf_idf_dic=dict()
    total_num_docs_sample=sample
    result_df=pd.DataFrame()
    for i in range(sample):
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0]+' '+df.city[0]
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
        result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),pd.Series(idf_calc.values)],axis=1)#.reset_index()
        result_df['tf_idf']=result_df[1]*result_df[2]

        for idx in range(result_df.shape[0]):
            tf_idf_dic[result_df[0][idx],i]=result_df['tf_idf'][idx]
    return tf_idf_dic        

In [45]:
# Second way--to check if it is the same like the 1st-for double checking the results
def calculate_tf_idf2(airbnb_data):
    idf_dic2={}
    tf_dic2={}
    proba={}
    for i in range(sample):
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0]+' '+df.city[0]
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
       
        for idx in range(len(tf_series)):
            idf_dic2[idf_series[idx],i]=idf_calc[idx] 
        for index,value in tf_series.iteritems():
            tf_dic2[index,i]=value
        for k in tf_dic2.keys():
            proba[k]=tf_dic2[k]*idf_dic2[k]
    return proba        

In [46]:
a=calculate_tf_idf(airbnb_data)

In [47]:
b=calculate_tf_idf2(airbnb_data)

In [48]:
for k in a.keys():
    if(a[k]!=b[k]):
        print('њет')

In [49]:
def compute_inverted_idx(doc_vocabs,vocabulary):
    """
    method that computes an inverted index
    
    input: doc_vocabs(dictionary), vocabulary(dictionary of all unique words, key=term, value=term_id)
    output: inverted_idx(dictionary, key=term_id, value=list of document_ids) 
    """
    #initialize defaultdict for making an inverted index
    inverted_idx = defaultdict(list)
    #in every document look for every word and assign document id to the words which belong to it
    for idx in doc_vocabs.keys():
        for word in doc_vocabs[idx]:
            inverted_idx[vocabulary[word]].append(idx)
    return inverted_idx

In [50]:
inverted_idx=compute_inverted_idx(doc_vocabs,vocabulary)

In [51]:
aa=('10', 0)
aa=aa[1]
aa

0

In [52]:
X=[1,2]
Y=[3,1]
cosine_similarity([X,Y])

array([[1.        , 0.70710678],
       [0.70710678, 1.        ]])

In [53]:
a2=defaultdict(list)
for k,v in a.items():
    k=k[1]
    a2[k].append(v)

In [54]:
a2

defaultdict(list,
            {0: [0.11808128682020748,
              0.11808128682020748,
              0.04126763878036154,
              0.11808128682020748,
              0.09261329264045663,
              0.05904064341010374,
              0.06174219509363775,
              0.04698926830123872,
              0.08253527756072308,
              0.017773004629742187,
              0.05904064341010374,
              0.05904064341010374,
              0.04126763878036154,
              0.11808128682020748,
              0.05904064341010374,
              0.11808128682020748,
              0.05904064341010374,
              0.09397853660247744,
              0.04126763878036154,
              0.035546009259484375,
              0.05904064341010374,
              0.05904064341010374,
              0.05904064341010374,
              0.05904064341010374,
              0.04126763878036154],
             1: [0.025147467381782817,
              0.03597789207803197,
              0.07195578415

In [55]:
cosine_similarity(0.92,0.3200323)

array([[1.]])

In [56]:
a=[0.05215743696136418,
  0.03257057712576686,
  0.032268990804419175]
#TFIDF_query
b=np.array([1.1446624 , 0.72522043, 0.74339473])
1 - spatial.distance.cosine(a, b)

NameError: name 'spatial' is not defined

In [ ]:
from scipy import spatial

dataSetI = [0.03597789207803197
           # ,0.03597789207803197,
            #  0.03597789207803197,
            #  0.03597789207803197
           ]
dataSetII = [0.03597789207803197#,
             # 0.03597789207803197,
             # 1.03597789207803197,
             # 2.03597789207803197
            ]
result = 1 - spatial.distance.cosine(dataSetI, dataSetII)

In [ ]:
result

In [ ]:
inverted_idx2=defaultdict(tuple)
for term_id in inverted_idx.keys():
    for k,v in vocabulary.items():
        if v==term_id:
            term=k
    for doc_id in range(len(inverted_idx[term_id])):
        inverted_idx[term_id].append(
            (doc_id,
             a[term,doc_id]))
        inverted_idx[term_id].remove(doc_id)

In [ ]:
inverted_idx

In [ ]:
for doc_id in inverted_idx[term_id]:
    print(doc_id)

In [ ]:
# Save a dictionary into a pickle file.
import pickle

pickle.dump(inverted_idx, open("inv_idx2_sample.p", "wb"))  # save it into a file named save.p

# Load the dictionary back from the pickle file.

inverted_index = pickle.load(open("inv_idx2_sample.p", "rb"))

# INVERTED INDEX 2

## 3.2.1) Inverted index

In [ ]:
tf_idf_dic=calculate_tf_idf(airbnb_data)

In [ ]:
tf_idf_dic

In [ ]:
inverted_idx2=defaultdict(list)
for term_id in inverted_idx.keys():
    for k,v in vocabulary.items():#k->term, v->term_id
        if v==term_id:
            term=k
    for doc_id in inverted_idx[term_id]:
        inverted_idx2[term_id].append((doc_id,tf_idf_dic[term,doc_id]))

In [ ]:
vocabulary['airport']

In [ ]:
inverted_idx2

In [ ]:
inverted_index[vocabulary['bathroom']]

In [ ]:
inverted_idx2[vocabulary['bathroom']]

In [ ]:
user_query="room bedroom"
#str(input())
user_query=preprocessing_text(user_query)
user_query_tfidf=np.ones(len(user_query))


list_term_idx=[]
#list of dataframes
list_tf_idf=[]

result_set=[]

result_tf_idf_dic=defaultdict(list)
for word in user_query:
    #if word exist in the vocabulary
    if word in vocabulary.keys():
        list_term_idx.append(set(inverted_idx[vocabulary[word]]))
        list_tf_idf.append((inverted_idx2[vocabulary[word]]))#[:,1])
        #result_tf_idf_dic
    else:
        list_term_idx.append({'x'})
        break
result_set=list(set.intersection(*list_term_idx))
if 'x' in result_set or not result_set:
    result_set='No results! Try again!'

tf_idf_dic=defaultdict(list)

for tf_idf_1doc in list_tf_idf:
    for tuple_pair in tf_idf_1doc:
        if tuple_pair[0] in result_set:
            tf_idf_dic[tuple_pair[0]].append(tuple_pair[1])

print(result_set)
#result_set=finalize_output(result_set)

In [ ]:
tf_idf_dic

In [ ]:
cosine_similarity_lst=[]
cosine_similarity_dic={}
for value in tf_idf_dic.values():
    print(cosine_similarity([user_query_tfidf],[value]))
    cosine_similarity_lst.append(cosine_similarity([user_query_tfidf],[value]))

In [ ]:
from heapq import heappush, heappop, nlargest,heapify

In [ ]:
heap = []
for item in cosine_similarity_lst:
     heappush(heap, item)
#heapify(cosine_similarity_lst)
#ordered = []
#while heap:
#    ordered.append(heappop(heap))
#ordered
#heap.sort(reverse=True)

In [ ]:
cosine_similarity_dic

In [ ]:
cosine_similarity_lst

In [ ]:
heap

In [ ]:
nlargest(3,heap)

In [ ]:
cosine_similarity_lst.hea

In [ ]:
1-cosine_similarity([np.array(a)],[np.array(b)])

In [ ]:
def cosine_distance(a, b):
    cosine_similarity=(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))
    return 1 - cosine_similarity

In [ ]:
cosine_distance([1,1,1],[0.03554600925948437,0.5,0.04])

In [ ]:
list_tf_idf

In [ ]:
res_tf_idf

In [ ]:
res_tf_idf

In [ ]:
inverted_idx2[14][0][0]

In [57]:
user_query.index('rooma')

NameError: name 'user_query' is not defined

In [58]:
a=pd.DataFrame(inverted_idx2[vocabulary['room']],columns=['doc_id','tf_idf'+str(user_query.index(word))])
a

NameError: name 'inverted_idx2' is not defined

In [59]:
c=[a,b]

In [60]:
b=pd.DataFrame(inverted_idx2[vocabulary['bedroom']],columns=['doc_id','tf_idf'+str(user_query.index(word))])
b

NameError: name 'inverted_idx2' is not defined

In [61]:
c=[a,b]

In [62]:
pd.merge(*c,left_on='doc_id',right_on='doc_id')

ValueError: can not merge DataFrame with instance of type <class 'list'>

In [63]:
np.array(inverted_idx2[23])

NameError: name 'inverted_idx2' is not defined

In [64]:
inverted_idx2

NameError: name 'inverted_idx2' is not defined

In [65]:
list_term_idx

NameError: name 'list_term_idx' is not defined

In [66]:
def finalize_output(result_set):
    df=pd.DataFrame()
    for i,val in enumerate(result_set):
        pd.set_option('display.max_colwidth', -1)
        df=df.append(pd.read_csv('data/doc_'+str(val)+'.tsv',sep='\t',usecols=['description','title','city','url']
                                 ,encoding='ISO-8859-1',index_col=False))
        df.reset_index().drop('index',axis=1)
    return df

In [67]:
def search_engine():
    user_query=str(input())
    
    user_query=preprocessing_text(user_query)

    list_term_idx=[]
    result_set=[]
    for word in user_query:
        #if word exist in the vocabulary
        if word in vocabulary.keys():
            list_term_idx.append(set(inverted_idx[vocabulary[word]]))
        else:
            list_term_idx.append({'x'})
            break
    result_set=list(set.intersection(*list_term_idx))
    if 'x' in result_set or not result_set:
        result_set='No results! Try again!'
        return result_set
        
    print(result_set)
    result_set=finalize_output(result_set)
    return result_set

In [68]:
search_engine()

TypeError: descriptor 'intersection' of 'set' object needs an argument

# more concise

# 3.2) Conjunctive query & Ranking score


## 3.2.1) Inverted index 

### Calculation of tf-idf values

In [ ]:
# First way
#TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

def calculate_tf_idf(airbnb_data):
    tf_idf_dic=dict()
    total_num_docs_sample=sample
    result_df=pd.DataFrame()
    for i in range(sample):
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0]+' '+df.city[0]
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
        result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),pd.Series(idf_calc.values)],axis=1)#.reset_index()
        result_df['tf_idf']=result_df[1]*result_df[2]

        for idx in range(result_df.shape[0]):
            tf_idf_dic[result_df[0][idx],i]=result_df['tf_idf'][idx]
    return tf_idf_dic        

In [ ]:
# Second way--to check if it is the same like the 1st-for double checking the results
def calculate_tf_idf2(airbnb_data):
    idf_dic2={}
    tf_dic2={}
    proba={}
    for i in range(sample):
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0]+' '+df.city[0]
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
       
        for idx in range(len(tf_series)):
            idf_dic2[idf_series[idx],i]=idf_calc[idx] 
        for index,value in tf_series.iteritems():
            tf_dic2[index,i]=value
        for k in tf_dic2.keys():
            proba[k]=tf_dic2[k]*idf_dic2[k]
    return proba        

In [ ]:
tf_idf_dic=calculate_tf_idf(airbnb_data)

In [ ]:
inverted_idx

In [ ]:
inverted_idx2=defaultdict(list)
for term_id in inverted_idx.keys():
    for k,v in vocabulary.items():#k->term, v->term_id
        if v==term_id:
            term=k
    for doc_id in inverted_idx[term_id]:
        inverted_idx2[term_id].append((doc_id,tf_idf_dic[term,doc_id]))

In [ ]:
inverted_idx2[vocabulary['bathroom']]

In [ ]:
tf_idf_dic

In [ ]:
cosine_similarity_lst=[]
for value in tf_idf_dic.values():
    tf_idf_val=cosine_similarity([user_query_tfidf],[value])[0][0]
    #keys()
    cosine_similarity_lst.append(tf_idf_val)

In [ ]:
tf_idf_dic

In [ ]:
cosine_similarity_lst

In [ ]:
from heapq import heappush, heappop, nlargest,heapify

In [ ]:
cosine_similarity_lst_tuples=[]
for key,value in tf_idf_dic.items():
    tf_idf_val=cosine_similarity([user_query_tfidf],[value])[0][0]
    cosine_similarity_lst_tuples.append((tf_idf_val,key))

In [ ]:
cosine_similarity_lst_tuples

In [69]:
def cosine_distance(a, b):
    cosine_similarity=(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))
    return 1 - cosine_similarity

In [70]:
cosine_distance([1,1,1],[0.03554600925948437,0.5,0.04])

0.3391904192171833

In [71]:
heap = []
for item in cosine_similarity_lst_tuples:
     heappush(heap, item)
#heap.sort(reverse=True)
nlargest(3,heap)

NameError: name 'cosine_similarity_lst_tuples' is not defined

In [72]:
(nlargest(3,heap))[1]

NameError: name 'nlargest' is not defined

## 3.2.2) Execute the query

In [444]:
user_query="room bedroom"
#str(input())
user_query=preprocessing_text(user_query)
user_query_tfidf=np.ones(len(user_query))

list_term_idx=[]
#list of dataframes
list_tf_idf=[]

result_set=[]

result_tf_idf_dic=defaultdict(list)
for word in user_query:
    #if word exist in the vocabulary
    if word in vocabulary.keys():
        list_term_idx.append(set(inverted_idx[vocabulary[word]]))
        list_tf_idf.append((inverted_idx2[vocabulary[word]]))#[:,1])
        #result_tf_idf_dic
    else:
        list_term_idx.append({'x'})
        break
result_set=list(set.intersection(*list_term_idx))
if 'x' in result_set or not result_set:
    result_set='No results! Try again!'

tf_idf_dic=defaultdict(list)

for tf_idf_1doc in list_tf_idf:
    for tuple_pair in tf_idf_1doc:
        if tuple_pair[0] in result_set:
            tf_idf_dic[tuple_pair[0]].append(tuple_pair[1])

print(result_set)
#result_set=finalize_output(result_set)

[0, 1, 3, 7]


In [451]:
wanted_doc_ids={}
for tup in (nlargest(3,heap)):
    wanted_doc_ids[tup[1]]=tup[0]

In [452]:
wanted_doc_ids

{1: 1.0, 3: 0.9486832980505138, 7: 1.0}

In [453]:
result_set=wanted_doc_ids.keys()
result_set

dict_keys([7, 1, 3])

In [454]:
result_set

dict_keys([7, 1, 3])

In [457]:
df=pd.DataFrame()

for i,val in enumerate(result_set):
    pd.set_option('display.max_colwidth', -1)
    df=df.append(pd.read_csv('data/doc_'+str(val)+'.tsv',sep='\t',usecols=['description','title','city','url']
                             ,encoding='ISO-8859-1',index_col=False))
    df.reset_index().drop('index',axis=1)

In [464]:
df['similarity']=wanted_doc_ids.values()

In [465]:
df[['title','description','city','url','similarity']]

,title,description,city,url,similarity
0,Friendly Private Room in ?Quiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX,1.000000
0,Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW  Alamo Heights Area. \n\nAmazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. \n\nFeaturing an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more\n\nEuropean inspired kitchen and top of the line decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX,1.000000
0,Private Room Close to Campus,Private bedroom in a cute little home situated in the coveted Garden Acres neighborhood in Bryan. The bedroom has its own private access and its own private bathroom.,Bryan,https://www.airbnb.com/rooms/11839729?location=College%20Station%2C%20TX,0.948683


In [432]:
#df.sort_values('Similarity',ascending=False)

# Final version

# 3.2) Conjunctive query & Ranking score


## 3.2.1) Inverted index 

### Calculation of tf-idf values

In [73]:
# First way
#TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

def calculate_tf_idf(airbnb_data):
    tf_idf_dic=dict()
    total_num_docs_sample=sample
    result_df=pd.DataFrame()
    for i in range(sample):
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0]+' '+df.city[0]
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
        result_df=pd.concat([pd.Series(idf_series.values),pd.Series(tf_series.values),pd.Series(idf_calc.values)],axis=1)#.reset_index()
        result_df['tf_idf']=result_df[1]*result_df[2]

        for idx in range(result_df.shape[0]):
            tf_idf_dic[result_df[0][idx],i]=result_df['tf_idf'][idx]
    return tf_idf_dic        

In [74]:
# Second way--to check if it is the same like the 1st-for double checking the results
def calculate_tf_idf2(airbnb_data):
    idf_dic2={}
    tf_dic2={}
    proba={}
    for i in range(sample):
        #take one file
        df=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',usecols=['description','title','city'],encoding='ISO-8859-1')
        #preprocessing 
        df=df.description[0]+' '+df.title[0]+' '+df.city[0]
        filtered_words=preprocessing_text(df)
        tf_series=pd.Series(filtered_words)
        tf_series=((tf_series.value_counts())/len(tf_series)).sort_index()
        idf_series=pd.Series(list(set(filtered_words))).sort_values()
        idf_calc=idf_series.apply(lambda x: np.log(total_num_docs_sample/len(inverted_idx[vocabulary[x]])))
       
        for idx in range(len(tf_series)):
            idf_dic2[idf_series[idx],i]=idf_calc[idx] 
        for index,value in tf_series.iteritems():
            tf_dic2[index,i]=value
        for k in tf_dic2.keys():
            proba[k]=tf_dic2[k]*idf_dic2[k]
    return proba        

In [75]:
tf_idf_dic=calculate_tf_idf(airbnb_data)

In [76]:
def compute_inverted_idx2(inverted_idx,vocabulary,tf_idf_dic):
    """
    method that computes the second inverted index
    
    input:  
    output: inverted_idx2(dictionary, key=term_id, value=list of tuples (document_id,tf_idf value) )
    """
    inverted_idx2=defaultdict(list)
    for term_id in inverted_idx.keys():
        for k,v in vocabulary.items():#k->term, v->term_id
            if v==term_id:
                term=k
        for doc_id in inverted_idx[term_id]:
            inverted_idx2[term_id].append((doc_id,tf_idf_dic[term,doc_id]))
    return inverted_idx2

In [77]:
inverted_idx2=compute_inverted_idx2(inverted_idx,vocabulary,tf_idf_dic)

In [78]:
inverted_idx2

defaultdict(list,
            {0: [(1, 0.025147467381782817), (4, 0.09754169166267275)],
             1: [(3, 0.10964690919019265)],
             2: [(6, 0.06578814551411559)],
             3: [(0, 0.11808128682020748)],
             4: [(1, 0.03597789207803197)],
             5: [(1, 0.018812075067592752),
              (4, 0.03648402437351322),
              (6, 0.03439922298074103)],
             6: [(0, 0.05904064341010374)],
             7: [(3, 0.10964690919019265)],
             8: [(6, 0.06578814551411559)],
             9: [(6, 0.06578814551411559)],
             10: [(5, 0.07939948596531193)],
             11: [(0, 0.06174219509363775),
              (6, 0.03439922298074103),
              (7, 0.042999028725926286)],
             12: [(4, 0.06977530584830442)],
             13: [(6, 0.06578814551411559)],
             14: [(1, 0.025147467381782817), (9, 0.20117973905426254)],
             15: [(0, 0.017773004629742187),
              (1, 0.010830424696249145),
              (

In [79]:
from heapq import heappush, heappop, nlargest,heapify

## 3.2.2) Execute the query

In [80]:
user_query="room bedroom"
#str(input())
user_query=preprocessing_text(user_query)
user_query_tfidf=np.ones(len(user_query))

list_term_idx=[]
#list of dataframes
list_tf_idf=[]

result_set=[]

result_tf_idf_dic=defaultdict(list)
for word in user_query:
    #if word exist in the vocabulary
    if word in vocabulary.keys():
        list_term_idx.append(set(inverted_idx[vocabulary[word]]))
        list_tf_idf.append((inverted_idx2[vocabulary[word]]))#[:,1])
        #result_tf_idf_dic
    else:
        list_term_idx.append({'x'})
        break
result_set=list(set.intersection(*list_term_idx))
if 'x' in result_set or not result_set:
    result_set='No results! Try again!'

tf_idf_dic=defaultdict(list)

for tf_idf_1doc in list_tf_idf:
    for tuple_pair in tf_idf_1doc:
        if tuple_pair[0] in result_set:
            tf_idf_dic[tuple_pair[0]].append(tuple_pair[1])

print(result_set)
#result_set=finalize_output(result_set)

[0, 1, 3, 7]


In [81]:
def cosine_sim_tuples(user_query_tfidf,tf_idf_dic):
    cosine_sim_lst_tuples=[]
    for key,value in tf_idf_dic.items():
        tf_idf_val=cosine_similarity([user_query_tfidf],[value])[0][0]
        cosine_sim_lst_tuples.append((tf_idf_val,key))
    return cosine_sim_lst_tuples

In [82]:
cosine_sim_lst_tuples=cosine_sim_tuples(user_query_tfidf,tf_idf_dic)

In [83]:
def heapify_tuples(cosine_sim_lst_tuples,k):
    heap = []
    for item in cosine_sim_lst_tuples:
         heappush(heap, item)
    return wanted_doc(nlargest(k,heap))

In [103]:
def wanted_doc(heap_k_docs):
    wanted_doc_ids={}
    for tup in (heap_k_docs):
        wanted_doc_ids[tup[1]]=round(tup[0],2)
    return wanted_doc_ids

In [104]:
wanted_doc_ids=heapify_tuples(cosine_sim_lst_tuples,3)

In [105]:
def cosine_distance(a, b):
    cosine_similarity=(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))
    return 1 - cosine_similarity

In [106]:
wanted_doc_ids

{1: 1.0, 3: 0.95, 7: 1.0}

In [107]:
result_set=wanted_doc_ids.keys()
result_set

dict_keys([7, 1, 3])

In [108]:
result_set

dict_keys([7, 1, 3])

In [109]:
df=pd.DataFrame()

for i,val in enumerate(result_set):
    pd.set_option('display.max_colwidth', -1)
    df=df.append(pd.read_csv('data/doc_'+str(val)+'.tsv',sep='\t',usecols=['description','title','city','url']
                             ,encoding='ISO-8859-1',index_col=False))
    df.reset_index().drop('index',axis=1)

In [110]:
df['similarity']=wanted_doc_ids.values()

In [111]:
df[['title','description','city','url','similarity']]

,title,description,city,url,similarity
0,Friendly Private Room in ?Quiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX,1.00
0,Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW  Alamo Heights Area. \n\nAmazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. \n\nFeaturing an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more\n\nEuropean inspired kitchen and top of the line decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX,1.00
0,Private Room Close to Campus,Private bedroom in a cute little home situated in the coveted Garden Acres neighborhood in Bryan. The bedroom has its own private access and its own private bathroom.,Bryan,https://www.airbnb.com/rooms/11839729?location=College%20Station%2C%20TX,0.95
